# How to run reV using HSDS
## Running reV Gen+Econ

[reV Gen](https://nrel.github.io/reV/_autosummary/reV.generation.generation.Gen.html#reV.generation.generation.Gen) and [reV Econ](https://nrel.github.io/reV/_autosummary/reV.econ.econ.Econ.html#reV.econ.econ.Econ) can be run locally using the [HDF Groups](https://www.hdfgroup.org) Highly Scalable Distributed Service (HSDS) to access resource .h5 file stored in the cloud (currenly on Amazon Web Services -- AWS).

reV uses [PySAM](https://nrel-pysam.readthedocs.io/en/latest/) to
compute technologically specific capcity factor means and profiles (`Gen`) and mean levelized cost of energy (`Econ`). `reV`
uses ``SAM`` technology terms and input configuration files.


NOTE: In all of these examples, the ``sam_file`` input points to files in
the reV test directory [`TESTDATADIR`](https://github.com/NREL/reV/tree/main/tests/data) that may not be copied in your install. You may want to download the relevant SAM system configs from that directory and point the ``sam_file`` variable to the correct filepath on your computer.

### Using HSDS

The Highly Scalable Distributed Service (HSDS) is a cloud optimized API to enable access to .h5 files hosted on [AWS](https://registry.opendata.aws). The HSDS software was developed by the [HDF Group](https://www.hdfgroup.org/) and is hosted on Amazon Web Services (AWS) using a combination of EC2 (Elastic Compute) and S3 (Scalable Storage Service). You can read more about the HSDS service [in this slide deck](https://www.slideshare.net/HDFEOS/hdf-cloud-services).


#### Setting up HSDS

To get started install the h5pyd library:

```bash
pip install h5pyd
```

Next, configure h5pyd by running ``hsconfigure`` from the command line, or by
creating a configuration file at ``~/.hscfg``:

```bash
hsconfigure

hs_endpoint = https://developer.nrel.gov/api/hsds
hs_username =
hs_password =
hs_api_key = {YOUR_API_KEY_HERE}
```

To get your own API key, visit https://developer.nrel.gov/signup/

Please note that our HSDS service is for demonstration purposes only. The API in the example above is hosted on an NREL server and will have limits on the amount of data you can access via HSDS. It is common to get an error: ``OSError: Error retrieving data: None errors`` if you attempt to access too much data or if the server is busy. Here are two references for scaling reV using HSDS and AWS:

1. [Setup your own HSDS server on your personal computer](https://nrel.github.io/rex/misc/examples.hsds.html#)
2. [Run reV on the AWS Parallel Cluster Infrastructure](https://nrel.github.io/reV/misc/examples.aws_pcluster.html)

#### Using HSDS with reV

Once h5pyd has been installed and configured, [rex](https://github.com/nrel/rex) can pull data directly from AWS using [HSDS](https://github.com/NREL/hsds-examples)

To access the resource data used by reV (NSRDB or WTK) you have to turn on the
``hsds`` flag in the `rex`'s [resource handlers](https://nrel.github.io/rex/rex/rex.renewable_resource.html):

In [1]:
import rex
from rex import init_logger

init_logger('reV', log_level='INFO')

nsrdb_file = '/nrel/nsrdb/v3/nsrdb_2013.h5'
with rex.Resource(nsrdb_file, hsds=True) as f:
    meta = f.meta
    time_index = f.time_index

display(meta.head())
display(time_index)

,latitude,longitude,elevation,timezone,country,state,county,urban,population,landcover
gid,,,,,,,,,,
0,-19.99,-175.259995,0.0,13,None,None,None,None,-9999,210
1,-19.99,-175.220001,0.0,13,None,None,None,None,-9999,210
2,-19.99,-175.179993,0.0,13,None,None,None,None,-9999,210
3,-19.99,-175.139999,0.0,13,None,None,None,None,-9999,210
4,-19.99,-175.100006,0.0,13,None,None,None,None,-9999,210


DatetimeIndex(['2013-01-01 00:00:00+00:00', '2013-01-01 00:30:00+00:00',
               '2013-01-01 01:00:00+00:00', '2013-01-01 01:30:00+00:00',
               '2013-01-01 02:00:00+00:00', '2013-01-01 02:30:00+00:00',
               '2013-01-01 03:00:00+00:00', '2013-01-01 03:30:00+00:00',
               '2013-01-01 04:00:00+00:00', '2013-01-01 04:30:00+00:00',
               ...
               '2013-12-31 19:00:00+00:00', '2013-12-31 19:30:00+00:00',
               '2013-12-31 20:00:00+00:00', '2013-12-31 20:30:00+00:00',
               '2013-12-31 21:00:00+00:00', '2013-12-31 21:30:00+00:00',
               '2013-12-31 22:00:00+00:00', '2013-12-31 22:30:00+00:00',
               '2013-12-31 23:00:00+00:00', '2013-12-31 23:30:00+00:00'],
              dtype='datetime64[ns, UTC]', length=17520, freq=None)

#### windpower

Compute wind capacity factors for a given set of latitude and longitude
coordinates:

In [2]:
import os
import numpy as np
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

lat_lons = np.array([[ 41.75, -73.66],
                     [ 40.05, -73.74],
                     [ 40.97, -73.78],
                     [ 41.65, -73.74],
                     [ 40.25, -73.7 ],
                     [ 41.05, -73.78]])

res_file = '/nrel/wtk/conus/wtk_conus_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/wind_gen_standard_losses_0.json')

pp = ProjectPoints.lat_lon_coords(lat_lons, res_file, sam_file)
gen = Gen('windpower', pp, sam_file, res_file,
          output_request=('cf_mean', 'cf_profile'))
gen.run(max_workers=1)
display(gen.out['cf_profile'])

INFO - 2021-09-21 14:56:33,065 [project_points.py:882] : Converting latitude longitude coordinates into nearest ProjectPoints


INFO:reV.config.project_points:Converting latitude longitude coordinates into nearest ProjectPoints


WARNING - 2021-09-21 14:56:48,482 [project_points.py:580] : WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"


INFO - 2021-09-21 14:56:48,493 [__init__.py:20] : Running with reV version 0.5.9


/home/gbuster/code/reV/reV/config/project_points.py:581: UserWarning: WARNING: points are not in sequential order and will be sorted! The original order is being preserved under column "points_order"
  warn(msg)
INFO:reV.generation.base:Running with reV version 0.5.9


...

INFO - 2021-09-21 14:56:51,753 [generation.py:536] : Running reV generation for: PointsControl for sites 2297790 through 2320057


INFO:reV.generation.generation:Running reV generation for: PointsControl for sites 2297790 through 2320057


INFO - 2021-09-21 14:57:55,032 [generation.py:550] : Finished reV gen serial compute for: PointsControl for sites 2297790 through 2320057


INFO:reV.generation.generation:Finished reV gen serial compute for: PointsControl for sites 2297790 through 2320057


array([[0.06 , 0.213, 0.265, 0.312, 0.83 , 0.827],
       [0.14 , 0.364, 0.438, 0.613, 0.831, 0.833],
       [0.023, 0.685, 0.371, 0.562, 0.833, 0.833],
       ...,
       [0.792, 0.589, 0.833, 0.833, 0.833, 0.833],
       [0.833, 0.833, 0.832, 0.798, 0.833, 0.833],
       [0.711, 0.827, 0.655, 0.19 , 0.833, 0.833]], dtype=float32)

#### pvwatts

NOTE: ``pvwattsv5`` and ``pvwattsv7`` are both available from reV.

Compute pv mean capacity factors and mean levelized cost of energy for all resource gids in New York county New York:

In [3]:
import os
from reV import TESTDATADIR
from reV.config.project_points import ProjectPoints
from reV.generation.generation import Gen

regions = {'Arlington': 'county'}

res_file = '/nrel/nsrdb/v3/nsrdb_2012.h5'  # HSDS 'file' path
sam_file = os.path.join(TESTDATADIR, 'SAM/naris_pv_1axis_inv13.json')

pp = ProjectPoints.regions(regions, res_file, sam_file)
gen = Gen('pvwattsv5', pp, sam_file, res_file,
          output_request=('cf_mean', 'lcoe_fcr'))
gen.run(max_workers=1)
display(gen.out['cf_mean'])
display(gen.out['lcoe_fcr'])

INFO - 2021-09-21 14:57:56,068 [project_points.py:964] : Extracting ProjectPoints for desired regions


INFO:reV.config.project_points:Extracting ProjectPoints for desired regions


INFO - 2021-09-21 14:58:20,158 [__init__.py:20] : Running with reV version 0.5.9


INFO:reV.generation.base:Running with reV version 0.5.9


INFO - 2021-09-21 14:58:20,165 [loggers.py:600] : Running with rex version 0.2.64


...

INFO:reV.generation.generation:Running reV generation for: PointsControl for sites 1142275 through 1144648


INFO - 2021-09-21 14:59:40,957 [generation.py:550] : Finished reV gen serial compute for: PointsControl for sites 1142275 through 1144648


INFO:reV.generation.generation:Finished reV gen serial compute for: PointsControl for sites 1142275 through 1144648


array([0.18077604, 0.17945854, 0.18129802, 0.17552263], dtype=float32)

array([128.7464 , 129.6916 , 128.37573, 132.59981], dtype=float32)